In [6]:
import rlepose.datasets.lxd_freihand as lxd_freihand
from rlepose.utils.config import update_config
cfg = update_config("./configs/256x192_res50_regress-flow_freihand.yaml")

train_dataset = builder.build_dataset(cfg.DATASET.TRAIN, preset_cfg=cfg.DATA_PRESET, train=True, heatmap2coord=cfg.TEST.HEATMAP2COORD)
print(f"load freihand dataset.")
train_sampler = torch.utils.data.distributed.DistributedSampler(
    train_dataset, num_replicas=opt.world_size, rank=opt.rank)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=cfg.TRAIN.BATCH_SIZE, shuffle=(train_sampler is None), num_workers=opt.nThreads, 
    sampler=train_sampler, worker_init_fn=_init_fn)
batch_size = 32
train_dataset = lxd_freihand.Freihand_CustomDataset(root_dir, split0_train, split1_train, split2_train,
                                            cfg, mode=mode_train)

TypeError: __init__() takes from 1 to 4 positional arguments but 6 were given

In [ ]:
import random
import numpy as np
import torch
import torch.multiprocessing as mp
import torch.utils.data

num_gpu = torch.cuda.device_count()


def _init_fn(worker_id):
    np.random.seed(1234)
    random.seed(1234)

train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset, num_replicas=16, rank=5)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=(train_sampler is None), num_workers=16, sampler=train_sampler, worker_init_fn=_init_fn)


In [ ]:
from rlepose.models import builder
device = torch.device('cuda')
def preset_model(cfg):
    model = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)

    if cfg.MODEL.PRETRAINED:
        model.load_state_dict(torch.load(cfg.MODEL.PRETRAINED))
    elif cfg.MODEL.TRY_LOAD:
        pretrained_state = torch.load(cfg.MODEL.TRY_LOAD)
        model_state = model.state_dict()
        pretrained_state = {k: v for k, v in pretrained_state.items()
                            if k in model_state and v.size() == model_state[k].size()}

        model_state.update(pretrained_state)
        model.load_state_dict(model_state)
    else:
        model._initialize()

    return model

m = preset_model(cfg)
m.cuda(device)

In [5]:

from rlepose.utils.metrics import DataLogger, calc_accuracy, calc_coord_accuracy, evaluate_mAP

m.train()
hm_shape = cfg.DATA_PRESET.get('HEATMAP_SIZE')
depth_dim = cfg.MODEL.get('DEPTH_DIM')
output_3d = cfg.DATA_PRESET.get('OUT_3D', False)
hm_shape = (hm_shape[1], hm_shape[0], depth_dim)
grad_clip = cfg.TRAIN.get('GRAD_CLIP', False)

for i, (inps, labels) in enumerate(train_loader):
        inps = inps.cuda()

        for k, _ in labels.items():
            if k == 'type':
                continue
            
            labels[k] = labels[k].cuda(device)

        output = m(inps, labels)

        prejts = output.pred_jts
        
        if i == 1:
             raise

NameError: name 'train_loader' is not defined

In [ ]:
torch.save(m.module.state_dict(), './exp/{}-{}/final.pth'.format(opt.exp_id, cfg.FILE_NAME))
print(f'Loading model from {opt.checkpoint}...')
# print('Loading model from {}...'.format(opt.checkpoint))
m.load_state_dict(torch.load(opt.checkpoint, map_location='cpu'), strict=True)  # 加载权重

检查valid函数的正确性

In [1]:
from rlepose.trainer import train, validate, validate_gt, validate_gt_3d

from rlepose.utils.transforms import get_coord

import rlepose.datasets.lxd_freihand as lxd_freihand
from rlepose.models import builder
from rlepose.utils.config import update_config

import torch
cfg = update_config("./configs/256x192_res50_regress-flow_freihand.yaml")

def preset_model(cfg):
    model = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)

    if cfg.MODEL.PRETRAINED:
        model.load_state_dict(torch.load(cfg.MODEL.PRETRAINED))
    elif cfg.MODEL.TRY_LOAD:
        pretrained_state = torch.load(cfg.MODEL.TRY_LOAD)
        model_state = model.state_dict()
        pretrained_state = {k: v for k, v in pretrained_state.items()
                            if k in model_state and v.size() == model_state[k].size()}

        model_state.update(pretrained_state)
        model.load_state_dict(model_state)
    else:
        model._initialize()

    return model

m = preset_model(cfg)
m.train()

hm_shape = cfg.DATA_PRESET.get('HEATMAP_SIZE')
depth_dim = cfg.MODEL.get('DEPTH_DIM')
output_3d = cfg.DATA_PRESET.get('OUT_3D', False)
hm_shape = (hm_shape[1], hm_shape[0], depth_dim)
grad_clip = cfg.TRAIN.get('GRAD_CLIP', False)



heatmap_to_coord = get_coord(cfg, cfg.DATA_PRESET.HEATMAP_SIZE, output_3d)

gt_val_dataset = builder.build_dataset(cfg.DATASET.VAL, preset_cfg=cfg.DATA_PRESET, train=False, heatmap2coord=cfg.TEST.HEATMAP2COORD)
gt_val_sampler = torch.utils.data.distributed.DistributedSampler(
    gt_val_dataset, num_replicas=cfg.TRAIN.WORLD_SIZE, rank=1)

gt_val_loader = torch.utils.data.DataLoader(
    gt_val_dataset, batch_size=32, shuffle=False, num_workers=20, drop_last=False, sampler=gt_val_sampler)


# gt_AP = validate_gt(m, opt, cfg, heatmap_to_coord)


build single from cfg.


/home/louxd/miniconda3/envs/deeplearning/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/louxd/miniconda3/envs/deeplearning/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


build single from cfg.
anno_xyz len： 3960
anno_K len： 3960
img list len： 3960
AUG OFF.


In [3]:
for inps, labels in gt_val_loader:
    bboxes = labels.pop('bbox')
    raise

RuntimeError: No active exception to reraise

In [4]:
bboxes.shape

torch.Size([32, 4])

In [55]:
import cv2
import numpy as np
import math
import torch
import torch
from rlepose.models import builder
from rlepose.utils.cal_mAP_lxd import cal_mAP_RMSE
cfg = update_config("./configs/256x192_res50_regress-flow_freihand.yaml")

def paint(im, kpts):
    '''
    输入为单帧的im与其对应的一组kpts
    '''
    
    colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
          [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], \
          [128, 0, 128], [255, 192, 203], [0, 128, 128], [128, 128, 0], [128, 0, 0], [0, 128, 0], [0, 0, 128]]
    limbSeq = [[0,1],[1,2],[2,3],[3,4],[0,5],[5,6],[6,7],[7,8],[0,9],[9,10],[10,11],[11,12],[0,13],[13,14],[14,15],[15,16],[0,17],[17,18],[18,19],[19,20]]

    # im = cv2.imread(img_path)
    # draw points
    for k in kpts:
        x = int(k[0])
        y = int(k[1])
        # print(f'im .shape = {im.shape}')
        # print(f'x={x}, y={y}')
        cv2.circle(im, (x, y), 2, (0, 0, 255), -1)

    # draw lines
    # print("kpts shape: ", kpts)
    for i in range(len(limbSeq)):
        cur_im = im.copy()
        limb = limbSeq[i]
        # print(kpts[limb[0]])
        [Y0, X0] = kpts[limb[0]]
        [Y1, X1] = kpts[limb[1]]
        mX = np.mean([X0, X1])
        mY = np.mean([Y0, Y1])
        length = ((X0 - X1) ** 2 + (Y0 - Y1) ** 2) ** 0.5
        angle = math.degrees(math.atan2(X0 - X1, Y0 - Y1))
        polygon = cv2.ellipse2Poly((int(mY), int(mX)), (int(length / 2), 4), int(angle), 0, 360, 1)
        cv2.fillConvexPoly(cur_im, polygon, colors[i])
        im = cv2.addWeighted(im, 0.4, cur_im, 0.6, 0)

    return im




device = torch.device('cuda')
m = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)  # 根据cfg的配置信息构建模型

m.load_state_dict(torch.load("/home/louxd/ldlib/res-loglikelihood-regression-master/weights/model_114.pth", map_location='cpu'), 
                  strict=True)  # 加载权重

m.cuda(device)  # 把模型放到gpu中

with torch.no_grad():
    gt_val_dataset = builder.build_dataset(cfg.DATASET.VAL, preset_cfg=cfg.DATA_PRESET, train=False, heatmap2coord=cfg.TEST.HEATMAP2COORD)
    gt_val_sampler = torch.utils.data.distributed.DistributedSampler(
        gt_val_dataset, num_replicas=1, rank=0)

    gt_val_loader = torch.utils.data.DataLoader(
        gt_val_dataset, batch_size=16, shuffle=False, num_workers=20, drop_last=False, sampler=gt_val_sampler)
    
    
    mAP_ary = []
    RMSE_ary = []
    for index, (inps, labels) in enumerate(gt_val_loader):
        inps = inps.cuda(device)

        for k, _ in labels.items():
            if k == 'type':
                continue
            
            labels[k] = labels[k].cuda(device)

        output = m(inps, labels)

        if isinstance(inps, list):
            batch_size = inps[0].size(0)
        else:
            batch_size = inps.size(0)

        kpts_gt = labels['target_uv'].cpu().numpy().reshape(-1, 21, 2) # size * 42
        kpts_pre = output.pred_jts.cpu().numpy().reshape(-1, 21, 2)

        mAP_str, RMSE = cal_mAP_RMSE(kpts_pre, kpts_gt)
        
        # print & draw
        for i in range(len(inps)):
            if i == 0 & index < 100:
                #print(f'batch:{batch_idx}_{i}')
                imgi = inps[i].cpu().numpy()
                imgi = np.transpose(imgi, (1, 2, 0))
                imgi = (imgi + np.array([0.480, 0.457, 0.406], dtype=np.float32)) * np.array([255., 255., 255.], dtype=np.float32)
                imgi = cv2.cvtColor(imgi, cv2.COLOR_BGR2RGB) 
                img_h, img_w, _ = imgi.shape
                kpts_pre_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_pre[i]])
                kpts_gt_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_gt[i]])

                imagei_pre = imgi.copy()
                imagei_gt = imgi.copy()
                imagei_pre = paint(imagei_pre, kpts_pre_i)
                imagei_gt = paint(imagei_gt, kpts_gt_i)
                cv2.imwrite(f'./exp/output_114/{index}_{i}_pre.jpg', imagei_pre)
                cv2.imwrite(f'./exp/output_114/{index}_{i}_gt.jpg', imagei_gt)

build single from cfg.
build single from cfg.
anno_xyz len： 3960
anno_K len： 3960
img list len： 3960
AUG OFF.


测试mAP计算模型

In [6]:
import torch
import cv2
import numpy as np
import torch
from rlepose.models import builder
from rlepose.utils.config import update_config

cfg = update_config("./configs/256x192_res50_regress-flow_freihand.yaml")
device = torch.device('cuda')
m = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)  # 根据cfg的配置信息构建模型

m.load_state_dict(torch.load("/home/louxd/ldlib/res-loglikelihood-regression-master/weights/model_114.pth", map_location='cpu'), 
                  strict=True)  # 加载权重

m.cuda(device)  # 把模型放到gpu中

with torch.no_grad():
    gt_val_dataset = builder.build_dataset(cfg.DATASET.VAL, preset_cfg=cfg.DATA_PRESET, train=False, heatmap2coord=cfg.TEST.HEATMAP2COORD)
    gt_val_sampler = torch.utils.data.distributed.DistributedSampler(
        gt_val_dataset, num_replicas=1, rank=0)

    gt_val_loader = torch.utils.data.DataLoader(
        gt_val_dataset, batch_size=16, shuffle=False, num_workers=20, drop_last=False, sampler=gt_val_sampler)
    
    
    rmse_list = []
    oks_list = []

    from rlepose.utils.valid_utils_lxd import compute_RMSE, calculate_oks_pt2, paint, calculate_mAP

    for index, (inps, labels) in enumerate(gt_val_loader):
        inps = inps.cuda(device)

        for k, _ in labels.items():
            if k == 'type':
                continue
            
            labels[k] = labels[k].cuda(device)

        output = m(inps, labels)

        if isinstance(inps, list):
            batch_size = inps[0].size(0)
        else:
            batch_size = inps.size(0)

        kpts_pre = labels['target_uv'].cpu().numpy().reshape(-1, 21, 2) # size * 42
        kpts_gt = output.pred_jts.cpu().numpy().reshape(-1, 21, 2)

        # valid & draw
        for i in range(len(inps)):

            imgi = inps[i].cpu().numpy()
            img_h, img_w, _ = imgi.shape
            kpts_pre_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_pre[i]])
            kpts_gt_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_gt[i]])

            rmse_i = compute_RMSE(kpts_pre_i, kpts_gt_i)
            oks_i = calculate_oks_pt2(kpts_pre_i, kpts_gt_i)

            rmse_list.append(rmse_i)
            oks_list.append(oks_i)

            if 0:
                print(f'********************* DRAW OUTPUT **********************')
                imgi = np.transpose(imgi, (1, 2, 0))
                imgi = (imgi + np.array([0.480, 0.457, 0.406], dtype=np.float32))  * np.array([255., 255., 255.], dtype=np.float32)
                imgi = cv2.cvtColor(imgi, cv2.COLOR_BGR2RGB) 
                imagei_pre = imgi.copy()
                imagei_gt = imgi.copy()
                imagei_pre = paint(imagei_pre, kpts_pre_i)
                imagei_gt = paint(imagei_gt, kpts_gt_i)
                cv2.imwrite(f'./exp/output_114/{index}_{i}_pre.jpg', imagei_pre)
                cv2.imwrite(f'./exp/output_114/{index}_{i}_gt.jpg', imagei_gt)

    rmse = sum(rmse_list) / len(rmse_list)
    mAP_info_str = calculate_mAP(oks_list)

    ap = mAP_info_str['AP']




anno_xyz len： 3960
anno_K len： 3960
img list len： 3960
AUG OFF.
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DRAW OUTPUT **********************
********************* DR

In [10]:
thresholds = np.arange(0.5, 1.0, 0.05)

In [11]:
thresholds

array([0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])